This script finds a human TAD for each human CNE and uses the homology between the TADs (generated by `compareTADsizes`) to generate a `lastz` job and search for the axolotl counterpart of the CNE in the respective axolotl TAD.

In [13]:
opts_tads = ['/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/rebuttal/TADs_and_CNEs/homologous_TADs.hg19_vs_amex.list']
opts_cnes = ['/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/rebuttal/TADs_and_CNEs/human_enhancers.hg19.fa']
opts_out = ['/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/compare_TADs/homologous_TADs.hg19_vs_amex.with_CNEs.list']

In [8]:
import pandas as pd
import os
import sys
import re

def loadCNEs(fastaFile):
    data = []
    with(open(fastaFile, 'r')) as hFile:
        for line in hFile.readlines():
            if not line.startswith('>'):
                continue
            chrName, start, end = line[1:].strip().split('_')
            data.append([chrName, int(start), int(end)])
    return pd.DataFrame(data=data, columns=['chr', 'start', 'end'])


In [26]:
cnes_hg = loadCNEs(opts_cnes[0])

# For each TAD, find the list of CNEs it overlaps
hOut = open(opts_out[0], 'w')
with(open(opts_tads[0], 'r')) as hFile:
    for line in hFile.readlines():
        cols = line.strip().split('\t')
        m = re.search('^([^:]+):([0-9]+)-([0-9]+)$', cols[0])
        if not m:
            print(f"ERROR: Malformatted line: '{line}'", file=sys.stderr)

        start = int(m.group(2))
        end = int(m.group(3))
        _cnes = cnes_hg[(cnes_hg['chr'] == m.group(1)) & (cnes_hg['start'] >= start) & (cnes_hg['end'] <= end)]
        if _cnes['chr'].count() == 0:
            cols.insert(1, 'N/A')
        else:
            cols.insert(1, str([f'{x[0]}:{x[1]}-{x[2]}' for x in _cnes.values]))
        print("\t".join(cols), file=hOut)
hOut.close()

In [11]:
print(cnes_hg.head())

    chr     start       end
0  chr1   3190581   3191428
1  chr1   8130439   8131887
2  chr1  10593123  10594209
3  chr1  10732070  10733118
4  chr1  10757664  10758631
